# Diseño de Prompts y Ejecución
1. Instalación `transformers`.
2. 

In [5]:
%pip install transformers
%pip install pandas
%pip install tensorflow
%pip install torch
%pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [3]:
from transformers import pipeline
import pandas as pd

In [4]:
df = pd.read_csv("input/yelp_review.csv")
reviews = df["texto"].tolist()

In [5]:
models = {
    "distilbert": pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english"),
    "bert": pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment"),
    "flan-t5": pipeline("text2text-generation", model="google/flan-t5-base")
}

c:\Users\claud\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\claud\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: Failed to import transformers.models.distilbert.modeling_tf_distilbert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [9]:
def zero_shot_prompt(text):
    return f"Classify the sentiment of this review as positive, negative or neutral: {text}"

def few_shot_prompt(text):
    return (
        "Review: I love this phone, it's fast and the camera is amazing.\nSentiment: positive\n"
        "Review: It broke after a week. Very bad quality.\nSentiment: negative\n"
        f"Review: {text}\nSentiment:"
    )

def cot_prompt(text):
    return (
        f"Analyze the review and decide if it's positive, negative or neutral.\n"
        f"Review: {text}\n"
        f"Step-by-step reasoning:"
    )

In [10]:
for name, model in models.items():
    print(f"\n=== {name.upper()} ===\n")
    for text in reviews[:3]:
        prompt = zero_shot_prompt(text)
        if name == "flan-t5":
            result = model(prompt, max_length=50)[0]['generated_text']
        else:
            result = model(prompt)[0]['label']
        print(f"Review: {text}\nPrediction: {result}\n")


=== DISTILBERT ===

Review: dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
Prediction: NEGATIVE

Review: Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don